<a href="https://colab.research.google.com/github/Dipali-Jadhav14/SQL_PROJECT_PAYROLL_SYSTEM/blob/main/MYSQL_(EMP_PAYSYSTEM)108.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mysql.connector
!pip install mysql-connector-python
!apt-get -y install mysql-server
!apt-get update --fix-missing
!service mysql start

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 79.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp312-cp312-linux_x86_64.whl size=247950 sha256=54eff2dba48833811ded01a507019b523e16e84f11b6ac22eba4dcb110f228d7
  Stored in directory: /root/.cache/pip/wheels/03/17/fa/d7604c72dd3dd6d3eb3d249abf36cc532c9a9b4354b8f1bc4f
Successfully built mysql.connector
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 40.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libaio1 libcgi-fast-perl libcgi-pm-perl libclone-perl libencode-locale-perl
  libfcgi-bin libfcgi-perl libfcgi0ldbl libhtml-parser-perl
  libhtml-tagset-perl libhtml-template-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl liblwp-mediatypes-perl libmecab2
  libprotobuf-lite23 liburi-perl mecab-ipadic mecab-ip

In [2]:

import mysql.connector

In [3]:
!service mysql start

 * Starting MySQL database server mysqld
   ...done.


In [4]:
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH 'mysql_native_password' BY '1411'; FLUSH PRIVILEGES;"

In [5]:
import mysql.connector

In [6]:
conn=mysql.connector.connect(
    user='root',
    password='1411',
    host='localhost')

In [7]:
cursor=  conn.cursor()

In [8]:
cursor.execute("CREATE DATABASE IF NOT EXISTS payroll_system")

#cursor.close()
#conn.close()



In [57]:
import mysql.connector

con = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1411"
)

cur = con.cursor()
cur.execute("USE payroll_system")

In [ ]:
!pip install mysql-connector-python

# ------------------------ Import Libraries ------------------------
import mysql.connector
from mysql.connector import Error

# ------------------------ Database Connection ------------------------
def create_connection():
    try:
        con = mysql.connector.connect(
            host="localhost",
            user="root",
            password="1411",
            database="payroll_system"
        )
        return con
    except Error as e:
        print("Error connecting to database:", e)
        return None

# ------------------------ Create Tables ------------------------
def create_tables():
    con = create_connection()
    if con is None:
        return
    cur = con.cursor()

    # Create new employee table
    cur.execute("""
    CREATE TABLE IF NOT EXISTS employee_info (
        emp_id INT PRIMARY KEY AUTO_INCREMENT,
        name VARCHAR(50),
        department VARCHAR(50),
        basic_salary DECIMAL(10,2),
        leaves INT DEFAULT 0,
        bonus DECIMAL(10,2) DEFAULT 0
    ) ENGINE=InnoDB
    """)

    # Create new payslip table
    cur.execute("""
    CREATE TABLE IF NOT EXISTS employee_payslip (
        slip_id INT PRIMARY KEY AUTO_INCREMENT,
        emp_id INT,
        gross_salary DECIMAL(10,2),
        total_deductions DECIMAL(10,2),
        net_salary DECIMAL(10,2),
        FOREIGN KEY (emp_id) REFERENCES employee_info(emp_id)
    ) ENGINE=InnoDB
    """)

    con.commit()
    cur.close()
    con.close()

# ------------------------ Add Employee ------------------------
def add_employee(name, dept, basic):
    con = create_connection()
    if con is None:
        return
    cur = con.cursor()
    query = "INSERT INTO employee_info (name, department, basic_salary) VALUES (%s,%s,%s)"
    cur.execute(query, (name, dept, basic))
    con.commit()
    emp_id = cur.lastrowid
    print(f"Employee added successfully with Employee ID: {emp_id}")
    cur.close()
    con.close()
    return emp_id

# ------------------------ Show Employees ------------------------
def show_employee():
    con = create_connection()
    if con is None:
        return
    cur = con.cursor()
    cur.execute("SELECT * FROM employee_info")
    rows = cur.fetchall()
    if not rows:
        print("No employees found")
    else:
        print("\n--- Employee List ---")
        print("ID | Name | Department | Basic | Leaves | Bonus")
        for row in rows:
            print(row)
    cur.close()
    con.close()

# ------------------------ Update Leaves & Bonus ------------------------
def update_leave_bonus(emp_id, leaves, bonus):
    con = create_connection()
    if con is None:
        return
    cur = con.cursor()
    cur.execute("SELECT * FROM employee_info WHERE emp_id=%s", (emp_id,))
    if cur.fetchone() is None:
        print("Employee ID does not exist!")
        cur.close()
        con.close()
        return
    cur.execute("UPDATE employee_info SET leaves=%s, bonus=%s WHERE emp_id=%s", (leaves, bonus, emp_id))
    con.commit()
    print("Leaves & Bonus Updated")
    cur.close()
    con.close()

# ------------------------ Generate Payslip ------------------------
def generate_payslip(emp_id):
    con = create_connection()
    if con is None:
        return
    cur = con.cursor()
    cur.execute("SELECT basic_salary, leaves, bonus FROM employee_info WHERE emp_id=%s", (emp_id,))
    data = cur.fetchone()
    if not data:
        print("Employee not found")
        cur.close()
        con.close()
        return
    basic_salary, leaves, bonus = data
    deduction_per_leave = 100
    deductions = leaves * deduction_per_leave
    gross_salary = basic_salary + bonus
    net_salary = gross_salary - deductions

    cur.execute("INSERT INTO employee_payslip (emp_id, gross_salary, total_deductions, net_salary) VALUES (%s,%s,%s,%s)",
                (emp_id, gross_salary, deductions, net_salary))
    con.commit()

    print("\n--- PAYSLIP ---")
    print(f"Employee ID: {emp_id}")
    print(f"Basic Salary: RS {basic_salary}")
    print(f"Bonus: RS {bonus}")
    print(f"Leave Deductions: RS {deductions}")
    print(f"Gross Salary: RS {gross_salary}")
    print(f"Net Salary: RS {net_salary}")

    cur.close()
    con.close()

# ------------------------ Menu ------------------------
def menu():
    create_tables()
    while True:
        print("\n----- PAYROLL SYSTEM -----")
        print("1. Add Employee")
        print("2. Show Employees")
        print("3. Update Leaves & Bonus")
        print("4. Generate Payslip")
        print("5. Exit")
        choice = input("Enter choice: ")

        if choice == '1':
            name = input("Enter employee name: ")
            dept = input("Enter department: ")
            basic = float(input("Enter basic salary: "))
            add_employee(name, dept, basic)

        elif choice == '2':
            show_employee()

        elif choice == '3':
            show_employee()
            emp_id = int(input("Enter employee ID to update: "))
            leaves = int(input("Enter number of leaves: "))
            bonus = float(input("Enter bonus amount: "))
            update_leave_bonus(emp_id, leaves, bonus)

        elif choice == '4':
            show_employee()
            emp_id = int(input("Enter employee ID for payslip: "))
            generate_payslip(emp_id)

        elif choice == '5':
            print("Exiting system")
            break

        else:
            print("Invalid choice")

# ------------------------ Run Program ------------------------
if __name__ == "__main__":
    menu()













----- PAYROLL SYSTEM -----
1. Add Employee
2. Show Employees
3. Update Leaves & Bonus
4. Generate Payslip
5. Exit
Enter choice: 1
Enter employee name: RICHA
Enter department: FINANCE
Enter basic salary: 30000
Employee added successfully with Employee ID: 2

----- PAYROLL SYSTEM -----
1. Add Employee
2. Show Employees
3. Update Leaves & Bonus
4. Generate Payslip
5. Exit
